## Libs usadas

> numpy:
Usar para manipulação aritmética de forma eficiente

> pandas:
Manipulação de arquivos csv, tabelas, estruturas de dados

> duckdb:
Roda os arquivos tudo em memória. Rápido e eficiente.

> parquet:
Armazenamento rápido e eficiente para grandes bases de dados

In [ ]:
!pip install duckdb pandas numpy pyarrow

In [ ]:
import pandas as pd
import duckdb as db
import pyarrow.parquet as pq
from pathlib import Path

In [ ]:
from google.colab import files
files.upload()

In [ ]:
df = pd.read_excel("recursos.xlsx")

In [ ]:
df.to_csv("converted_recursos.csv", index=False)

In [ ]:
!ls

converted_recursos.csv		  drive
disponibilidade-jan-abr-2025.csv  recursos.xlsx
disponibilidade-mai-ago-2025.csv  sample_data


In [ ]:
data_csv = {
    "disponibilidade1": "disponibilidade-jan-abr-2025.csv",
    "disponibilidade2": "disponibilidade-mai-ago-2025.csv",
    "recursos": "converted_recursos.csv"
}

In [ ]:
for nome, csv_path in data_csv.items():
  parquet_path = Path(csv_path).with_suffix(".parquet")

  # fazer leitura do CSV com pandas
  df = pd.read_csv(csv_path, encoding='latin1') #mudar o enconding

  # Salvar agora como parquet
                #caminho do parquet
  df.to_parquet(parquet_path, engine="pyarrow", compression="zstd")

  # se deu certo vamos ver
  print(f"{csv_path} -> {parquet_path}")

disponibilidade-jan-abr-2025.csv -> disponibilidade-jan-abr-2025.parquet
disponibilidade-mai-ago-2025.csv -> disponibilidade-mai-ago-2025.parquet
converted_recursos.csv -> converted_recursos.parquet


In [ ]:
!ls

In [ ]:
con = db.connect()

for nome, csv_path in data_csv.items():
  parquet_path = Path(csv_path).with_suffix(".parquet")

  con.execute(f"""
        CREATE OR REPLACE TABLE "{nome}" AS
        SELECT * FROM read_parquet('{parquet_path}')
    """)

In [ ]:
# Exemplo: contar registros em cada tabela
for name in data_csv.keys():
    result = con.execute(f"SELECT COUNT(*) AS total FROM {name}").fetchdf()
    print(name, result)


disponibilidade1     total
0  732168
disponibilidade2     total
0  771264
recursos    total
0  34396
